# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.6.0
Keras Version: 2.6.0

Python 3.8.0 (default, Nov  6 2019, 16:00:02) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.5
Scikit-Learn 1.0.2
GPU is available


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

import os

In [3]:
classifier = Sequential()

#classes = 2

#Kernels usually 1x1 | 3x3 | 5x5

#Adding a convolutional layer
classifier.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.40))

#Adding a second convolutional layer
classifier.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.40))

#Adding a third convolutional layer
classifier.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.40))

#Flattening
classifier.add(Flatten())


#Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.30))
classifier.add(Dense(units = 1, activation= 'sigmoid'))

Let's double check the model description:

In [4]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        3

In [5]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path_dir = '../dataset_mel_final/train'
parth_dir_test = '../dataset_mel_final/test'

datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    shuffle=True,
    subset='training',
    batch_size = 64,
    class_mode = 'binary'
)
validation_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    subset='validation',
    batch_size = 64,
    class_mode = 'binary'
)

test_set = datagen.flow_from_directory(parth_dir_test,
                                        target_size=(150,150),
                                        batch_size =64,
                                        class_mode = 'binary')


Found 1102 images belonging to 2 classes.
Found 274 images belonging to 2 classes.
Found 344 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'nonviolence': 0, 'violence': 1}

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=50)
mc = ModelCheckpoint('best_model2.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

# Create a TensorBoard callback
log_dir = "logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
classifier.fit(train_generator,
                         epochs = 600,
                         callbacks = [tensorboard_callback,es,mc,reduceLR],
                         validation_data = validation_generator
                         )

Epoch 1/600
18/18 [==============================] - 10s 323ms/step - loss: 0.8053 - accuracy: 0.5082 - val_loss: 0.6927 - val_accuracy: 0.5000
Epoch 2/600
18/18 [==============================] - 4s 225ms/step - loss: 0.6889 - accuracy: 0.5245 - val_loss: 0.6801 - val_accuracy: 0.6861
Epoch 3/600
18/18 [==============================] - 4s 238ms/step - loss: 0.6483 - accuracy: 0.6361 - val_loss: 0.6229 - val_accuracy: 0.7153
Epoch 4/600
18/18 [==============================] - 4s 226ms/step - loss: 0.5855 - accuracy: 0.6770 - val_loss: 0.5246 - val_accuracy: 0.7372
Epoch 5/600
18/18 [==============================] - 4s 224ms/step - loss: 0.4854 - accuracy: 0.7722 - val_loss: 0.3845 - val_accuracy: 0.8504
Epoch 6/600
18/18 [==============================] - 4s 228ms/step - loss: 0.3847 - accuracy: 0.8494 - val_loss: 0.2995 - val_accuracy: 0.8869
Epoch 7/600
18/18 [==============================] - 4s 220ms/step - loss: 0.3036 - accuracy: 0.8802 - val_loss: 0.2453 - val_accuracy: 0.927

Epoch 58/600
18/18 [==============================] - 4s 217ms/step - loss: 0.0811 - accuracy: 0.9691 - val_loss: 0.1243 - val_accuracy: 0.9526
Epoch 59/600
18/18 [==============================] - 4s 220ms/step - loss: 0.0824 - accuracy: 0.9664 - val_loss: 0.1243 - val_accuracy: 0.9526
Epoch 60/600
18/18 [==============================] - 4s 215ms/step - loss: 0.0870 - accuracy: 0.9664 - val_loss: 0.1243 - val_accuracy: 0.9526
Epoch 61/600
18/18 [==============================] - 4s 214ms/step - loss: 0.0689 - accuracy: 0.9755 - val_loss: 0.1243 - val_accuracy: 0.9526
Epoch 62/600
18/18 [==============================] - 4s 214ms/step - loss: 0.0714 - accuracy: 0.9773 - val_loss: 0.1243 - val_accuracy: 0.9526
Epoch 63/600
18/18 [==============================] - 4s 214ms/step - loss: 0.0857 - accuracy: 0.9737 - val_loss: 0.1243 - val_accuracy: 0.9526
Epoch 64/600
18/18 [==============================] - 4s 215ms/step - loss: 0.0786 - accuracy: 0.9737 - val_loss: 0.1243 - val_accuracy:

In [ ]:
#Overfitting -> Your training accuracy will be higher than the accuracy on the validation/test set

#Overfitting indicates that your model is too complex for the problem that it is solving,
#filters in the case of Convolutional Neural Networks, and layers in the case of overall Deep Learning Models

#How do you know if your model is underfitting? Your model is underfitting if the accuracy on the validation set 
#is higher than the accuracy on the training set. Additionally, 
#if the whole model performs bad this is also called underfitting.

In [10]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 11704), started 22:45:26 ago. (Use '!kill 11704' to kill it.)

# Part 3 test the CNN

In [11]:
# Load Model
classifier.load_weights('best_model2.h5')

In [12]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        3

In [13]:
score = classifier.evaluate(test_set, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

Test Loss 0.20216502249240875
Test accuracy 0.9331395626068115
